In [1]:
import pandas as pd
from api_keys import weather_api_key
import requests
import calendar
# Impoirt cab rides data
cab_rides = pd.read_csv("cab_rides.csv/cab_rides.csv")
cab_rides.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL


In [2]:
#select uber rides only
uber_rides_data = cab_rides.loc[cab_rides["cab_type"] == "Uber"]
uber_rides_data.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
12,1.11,Uber,1543615981179,West End,North End,12.0,1.0,009e9c53-074d-43cf-aef2-0fbc7a47ed3d,6f72dfc5-27f1-42e8-84db-ccc7a75f6969,UberXL
13,1.11,Uber,1544698211014,West End,North End,16.0,1.0,23f145da-f0c1-4d1f-a184-496bc003a7db,6c84fd89-3f11-4782-9b50-97c468b19529,Black
14,1.11,Uber,1544728503935,West End,North End,7.5,1.0,357559cb-8c58-4278-a41a-e33b2e0997a3,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX
15,1.11,Uber,1545004511430,West End,North End,7.5,1.0,50ef1165-9d23-416c-a65c-18906207b295,9a0e7b09-b92b-4c41-9779-2ad22b4d779d,WAV
16,1.11,Uber,1544748007961,West End,North End,26.0,1.0,91c4861c-1780-42b0-bca1-bbd64a422cc3,6d318bcc-22a3-4af6-bddd-b409bfce1546,Black SUV


In [3]:
uber_rides_data["name"].unique()

array(['UberXL', 'Black', 'UberX', 'WAV', 'Black SUV', 'UberPool', 'Taxi'],
      dtype=object)

In [3]:
# select relevant columns, drop na, set index, import to csv
uber_rides = uber_rides_data[["id", "distance", "source", "destination", "price", "time_stamp", "name"]]
uber_rides.dropna(inplace = True)
uber_rides.set_index("id", inplace = True)
uber_rides.head()
uber_rides_random = uber_rides.sample(n=5000, random_state=1)
# uber_rides_random.to_csv("uber_rides_random.csv")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
boston_weather = pd.read_csv("boston-weather.csv")
# boston_weather_nov18 = boston_weather.loc[(boston_weather["Year"] == 2018)]
# boston_weather.tail()

date = boston_weather["Day"]
year = []
month = []
days = []

for day in date.tolist():
    split_date = day.split()
    month.append(split_date[1])
    year.append(split_date[2])
    days.append(split_date[0])

    
boston_weather["Month"] = month
boston_weather["Year"] = year
boston_weather["Days"] = days

d = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun': 6, 'jul': 7, 'aug':8,'sep': 9,'oct': 10,'nov':11,'dec': 12 }
boston_weather.Month = boston_weather.Month.map(d)

# boston_weather_dec18 = boston_weather.loc[(boston_weather["Year"] == "2018") & (boston_weather["Month"] == "dec")]
# cleaned_boston_weather_dec18 = boston_weather_dec18.replace("T", "0.01")
# cleaned_boston_weather_dec18
boston_weather.tail(50)
boston_weather["Date"] = boston_weather["Days"].astype(str) + boston_weather["Month"].astype(str)+ boston_weather["Year"].astype(str)
boston_weather = boston_weather[["Day", "High(°F)", "Low(°F)", "Precip.(inch)","Month", "Year", "Days", "Date"]]
boston_weather.rename(columns = {'Precip.(inch)':'Precip','High(°F)': 'High', 'Low(°F)': 'Low'}, inplace = True) 
boston_weather = boston_weather[boston_weather.Precip != "T"]
boston_weather

,Day,High,Low,Precip,Month,Year,Days,Date
0,1 jan 2015,33.1,22.1,0.00,1,2015,1,112015
1,2 jan 2015,41.0,31.1,0.00,1,2015,2,212015
2,3 jan 2015,37.0,22.1,0.62,1,2015,3,312015
3,4 jan 2015,52.0,35.1,0.57,1,2015,4,412015
4,5 jan 2015,50.0,17.2,0.00,1,2015,5,512015
...,...,...,...,...,...,...,...,...
1455,27 dec 2018,37.9,30.2,0.00,12,2018,27,27122018
1456,28 dec 2018,57.0,34.0,0.33,12,2018,28,28122018
1457,29 dec 2018,54.0,32.0,0.00,12,2018,29,29122018
1458,30 dec 2018,34.0,26.2,0.00,12,2018,30,30122018


In [6]:
import datetime

for index, row in uber_rides_random.iterrows():
    time = row["time_stamp"]
    
    new_time = str(datetime.datetime.fromtimestamp(time/1000).strftime('%d-%m-%Y'))
    
    try:
        uber_rides_random.loc[index,"new_time"] = new_time
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [7]:
uber_rides_random

,distance,source,destination,price,time_stamp,name,new_time
id,,,,,,,
63f50349-a66e-4f42-a41b-6d8fbb7b5bd5,1.50,Fenway,Back Bay,7.0,1543417583188,WAV,28-11-2018
a646a0bd-aed1-4df7-941c-08080b0614ee,3.79,Northeastern University,Financial District,11.5,1543417328332,UberX,28-11-2018
39c334fd-75d0-4acc-bef7-ea7eaa1a414d,0.54,South Station,Financial District,5.5,1545153005565,UberPool,18-12-2018
39c64126-a614-4ffb-ba1c-a4716483f374,1.16,Haymarket Square,Theatre District,17.0,1543320202160,Black,27-11-2018
407b2fe4-6ca2-4760-b883-957cb4e333e0,1.81,Theatre District,Northeastern University,16.5,1543430827626,Black,28-11-2018
...,...,...,...,...,...,...,...
a811d822-97db-4536-b8ae-2ed40ba7f6f8,1.56,Boston University,Back Bay,8.5,1544976911577,WAV,16-12-2018
3858671a-f353-425c-ab26-2272ed0aa557,1.35,Beacon Hill,North End,14.0,1545068414904,UberX,17-12-2018
3c5e31fa-3c74-4921-ad85-617660701e4f,4.49,Northeastern University,Financial District,25.0,1543252811042,Black,26-11-2018


In [ ]:
uber_date = uber_rides_random["new_time"]
uber_year = []
uber_month = []
uber_days = []

for uber_day in uber_date.tolist():
    split_uber_date = uber_day.split("-")
    uber_days.append(split_uber_date[0])
    uber_month.append(split_uber_date[1])
    uber_year.append(split_uber_date[2])
    
    
uber_rides_random["Days"] = uber_days
uber_rides_random["Month"] = uber_month
uber_rides_random["Year"] = uber_year
uber_rides_random
uber_rides_random["Date"] = uber_rides_random["Days"].astype(str) + uber_rides_random["Month"].astype(str)+ uber_rides_random["Year"].astype(str)
uber_rides_random = uber_rides_random.drop(columns=['time_stamp'])

In [19]:

uber_rides_random.to_csv("uber_rides_random.csv")
boston_weather.to_csv("boston_weather.csv",encoding = "utf-8")

In [ ]:
cleaned_boston_weather_nov18 = boston_weather_nov18.replace("T", "0.01")
cleaned_boston_weather_nov18

In [ ]:
uber_rides_data["destination"].nunique()

In [ ]:
uber_rides_data["source"].nunique()

In [ ]:
destination = uber_rides_data["destination"]

In [ ]:
source = uber_rides_data["source"]

location = pd.concat([source, destination])
location

In [ ]:
uber_map = pd.DataFrame(location.unique())
#uber_map = pd.DataFrame(uber_rides_data["source"].unique())

In [ ]:
uber_map

In [ ]:
uber_map = uber_map.rename(columns={0: "location"})
uber_map["lat"] = ""
uber_map["long"] = ""
uber_map